In [3]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Bidirectional, GRU, Dense, BatchNormalization, Dropout, LeakyReLU
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import load_img, img_to_array

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train = x_train / 255.0
x_test = x_test / 255.0
y_train = to_categorical(y_train, num_classes=10)
y_test = to_categorical(y_test, num_classes=10)

timesteps = x_train.shape[1]
input_dim = x_train.shape[2]

model = Sequential([
    Bidirectional(GRU(128, return_sequences=True, dropout=0.2), input_shape=(timesteps, input_dim)),
    BatchNormalization(),
    Bidirectional(GRU(64, dropout=0.2)),
    Dense(64),
    LeakyReLU(alpha=0.1),
    Dropout(0.3),
    Dense(10, activation='softmax')
])

model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])
model.fit(x_train, y_train, epochs=2, batch_size=64, validation_split=0.2)
test_loss, test_accuracy = model.evaluate(x_test, y_test, verbose=2)
print(f"Test Accuracy: {test_accuracy:.2f}")

def predict_image(image_path):
    img = load_img(image_path, color_mode='grayscale', target_size=(28, 28))
    img = img_to_array(img) / 255.0
    img = np.mean(img, axis=-1)
    img = 1 - img
    img = img.reshape(1, 28, 28)
    prediction = model.predict(img)
    top_3_indices = np.argsort(prediction[0])[-3:][::-1]
    top_3_probs = prediction[0][top_3_indices]
    for i, (digit, prob) in enumerate(zip(top_3_indices, top_3_probs), 1):
        print(f"Top {i}: Digit {digit} with Probability: {prob * 100:.2f}%")

image_path = "digit.png"
predict_image(image_path)


Epoch 1/2
750/750 ━━━━━━━━━━━━━━━━━━━━ 162s 202ms/step - accuracy: 0.7038 - loss: 0.8652 - val_accuracy: 0.9672 - val_loss: 0.1073
Epoch 2/2
750/750 ━━━━━━━━━━━━━━━━━━━━ 203s 202ms/step - accuracy: 0.9570 - loss: 0.1459 - val_accuracy: 0.9784 - val_loss: 0.0766
313/313 - 7s - 21ms/step - accuracy: 0.9782 - loss: 0.0682
Test Accuracy: 0.98
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
Top 1: Digit 6 with Probability: 99.95%
Top 2: Digit 0 with Probability: 0.02%
Top 3: Digit 4 with Probability: 0.01%
